## 텍스트 데이터의 정제와 정규화
---

- 정제(cleaning): 갖고 있는 Corpus(말뭉치)로부터 노이지 데이터를 제거한다.
- 정규화(onrmalization): 표현 방법이 다른 단어들을 통합시켜서 같은 단어로 만들어준다.

#### 1. 규칙에 기반한 표기가 다른 단어들을 통합
- ex) US와 USA는 같은 의미를 가진 단어이기 때문에 단어를 통합한다.

#### 2. 대소문자 통합
- 대부분 대문자를 소문자롭 변환한다.
- 고유명사를 구분하기 위해서는 대문자가 유지되어야 함
- 문장의 맨 앞 글자만 소문자로 변환하는 방법이 있음
- 예외 상황을 고려하지 않고 소문자로 변환하는게 효율적인 경우도 있음

#### 3. 불필요한 단어 제거
- 불용어, 빈도가 적은 단어, 길이가 짧은 단어를 제거하는 방법이 있음.

In [2]:
# 길이가 1~2인 단어들을 정규 표현식을 이용하여 삭제

import re

text = " I was wondering if anyone out there could enlighten me on this car."

shortword = re.compile(r'\W*\b\w{1,2}\b')
print(shortword.sub('', text))

 was wondering anyone out there could enlighten this car.


## 불용어(Stopwords)
---
#### 불용어: 자주 등장하지만 의미에 크기 기여하지 않는 단어들

In [4]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from konlpy.tag import Okt

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dltkd\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [10]:
# NLTK에서 불용어 확인

stop_words_list = stopwords.words('english')

print("불용어 개수: ", len(stop_words_list))
print("불용어 10개 출력: ", stop_words_list[-10:])

불용어 개수:  198
불용어 10개 출력:  ['y', 'you', "you'd", "you'll", 'your', "you're", 'yours', 'yourself', 'yourselves', "you've"]


In [11]:
# NLTK를 통해 불용어 제거하기

sent = "Family is not an important thing. It's everything"
stop_words = set(stopwords.words('english'))

word_tokens = word_tokenize(sent)

result = []
for word in word_tokens:
    if word not in stop_words:
        result.append(word)

print("불용어 제거 전: ", word_tokens)
print("불용어 제거 후: ", result)

불용어 제거 전:  ['Family', 'is', 'not', 'an', 'important', 'thing', '.', 'It', "'s", 'everything']
불용어 제거 후:  ['Family', 'important', 'thing', '.', 'It', "'s", 'everything']


In [13]:
# 한국어에서 불용어 제거

okt = Okt()

sent = "고기를 아무렇게나 구우려고 하면 안돼. 고기라고 다 같은 게 아니거든. 예컨대 삼겹살을 구울 때는 중요한 게 있지."
stop_words = "를 아무렇게나 구 우려 고 안 돼 같은 게 구울 때 는"

stop_words = set(stop_words.split(' '))
word_tokens = okt.morphs(sent)

result = [word for word in word_tokens if not word in stop_words]

print('불용어 제거 전: ', word_tokens)
print("불용어 제거 후: ", result)


불용어 제거 전:  ['고기', '를', '아무렇게나', '구', '우려', '고', '하면', '안', '돼', '.', '고기', '라고', '다', '같은', '게', '아니거든', '.', '예컨대', '삼겹살', '을', '구울', '때', '는', '중요한', '게', '있지', '.']
불용어 제거 후:  ['고기', '하면', '.', '고기', '라고', '다', '아니거든', '.', '예컨대', '삼겹살', '을', '중요한', '있지', '.']


## 정규 표현식
- re모듈 사용
- 정규 표현식 문법

특수 문자|설명
-|-
.|한 개의 임의의 문자를 나타냄(줄바꿈 문자인 \n은 제외)
?|앞의 문자가 존재할 수도 있고, 존재하지 않을 수도 있음(0개 or 1개)
*|앞의 문자가 무한개로 존재할 수도 있고, 존재하지 않을 수도 있음(0개 이상)
+|앞의 문자가 최소 한개 이상 존재
^|뒤의 문자열로 문자열이 시작됨
$|앞의 문자열로 문자열이 끝남
{숫자}|숫자만큼 반복
{숫자1,숫자2}|숫자1이상 숫자 2이하만큼 반복
{숫자,}|숫자 이상만큼 반복
[]|대괄호 안의 문자들 중 한개의 문자와 매치([a-z] 소문자 전체, [a-zA-Z] 알파벳 전체)
[^문자]|해당 문자를 제외한 문자 매치
l|AlB와 같이 쓰이며 A 또는 B의 의미를 가짐

- 역슬래쉬를 사용한 문법

문자 규칙|설명
-|-
\\\\\ |역 슬래쉬 문자 자체를 의미
\\\d| 모든 숫자를 의미 [0-9]와 동일
\\\D| 숫자를 제외한 모든 문자를 의미 [^0-9]와 동일
\\\s| 공백을 의미 [ \\t\\n\\r\\f\\v]와 동일
\\\S| 공백을 제외한 문자를 의미 [^ \\t\\n\\r\\f\\v]와 동일
\\\w| 문자 또는 숫자를 의미 [a-zA-Z0-9]와 동일
\\\W| 문자 또는 숫자가 아닌 문자를 의미 [^a-zA-Z0-9]와 동일

- re모듈 함수

모듈 함수|설명
-|-
re.compile()|정규표현식을 컴파일하는 함수
re.search()|문자열 전체에 대해서 정규표현식과 매치되는지 검색
re.match()|문자열의 처음이 정규표현식과 매치되는지 검색
re.split()|정규 표현식을 기준으로 문자열을 분리하여 리스트로 리턴
re.findall()|문자열과 정규표현식이 매치되는 모든 경우의 문자열을 리스트로 반환
re.finditer()|문자열에서 정규표현식과 매치되는 문자열을 이터레이터 객체로 반환
re.sub()|문자열과 정규표현식이 일치하는 부분에 대해서 다른 문자열로 대채

In [14]:
import re

In [ ]:
# .기호
# a.c는 a와 c사이에 한개의 아무 문자가 올 수 있음
r = re.compile("a.c")
print(r.search("kkk"))
print(r.search("abc"))

None
<re.Match object; span=(0, 3), match='abc'>


In [ ]:
# ?기호
# ab?c는 b가 있을수도 있고 없을수도 있고
r = re.compile("ab?c")
print(r.search("abbc"))
print(r.search("abc"))

None
<re.Match object; span=(0, 3), match='abc'>


In [18]:
# *기호
# ab*c는 b와c 사이에 뭐든 가능
r = re.compile("ab*c")
print(r.search("a"))
print(r.search("ac"))
print(r.search("abc"))

None
<re.Match object; span=(0, 2), match='ac'>
<re.Match object; span=(0, 3), match='abc'>


In [19]:
# +기호
# ab+c는 abc, abbc, abbbc등 +앞 기호가 하나 이상 와야함
r = re.compile("ab+c")
print(r.search("ac"))
print(r.search("abbbbc"))

None
<re.Match object; span=(0, 6), match='abbbbc'>


In [20]:
# ^기호
# ^ab는 ab로 시작해야함
r = re.compile("^ab")
print(r.search("bbc"))
print(r.search("zab"))
print(r.search("abc"))

None
None
<re.Match object; span=(0, 2), match='ab'>


In [22]:
# {숫자} 기호
# ab{2}c 는 a와 c사이에 b가 2개인 문자열
r = re.compile("ab{2}c")
print(r.search("ac"))
print(r.search("abc"))
print(r.search("abbbbc"))
print(r.search("abbc"))

None
None
None
<re.Match object; span=(0, 4), match='abbc'>


In [23]:
# {숫자1, 숫자2} 기호
# ab{2,8}c는 b가 2와8 사이의 개수가 와야함
r = re.compile("ab{2,8}c")

print(r.search("abbbbbbbc"))

<re.Match object; span=(0, 9), match='abbbbbbbc'>


#### 정규 표현식 텍스트 전처리 예제

In [24]:
text = """100 John      PROF
101 James       STUD
102 Mac     STUD"""

In [25]:
#\s+는 공백을 찾아내는 정규표현식
re.split('\s+', text)

<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
C:\Users\dltkd\AppData\Local\Temp\ipykernel_3932\4215893576.py:2: SyntaxWarning: invalid escape sequence '\s'
  re.split('\s+', text)


['100', 'John', 'PROF', '101', 'James', 'STUD', '102', 'Mac', 'STUD']

In [26]:
# \d는 숫자에 해당하는 정규표현식
re.findall('\d+', text)

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\dltkd\AppData\Local\Temp\ipykernel_3932\2617481547.py:2: SyntaxWarning: invalid escape sequence '\d'
  re.findall('\d+', text)


['100', '101', '102']

In [27]:
re.findall('[A-Z]', text)

['J', 'P', 'R', 'O', 'F', 'J', 'S', 'T', 'U', 'D', 'M', 'S', 'T', 'U', 'D']

In [32]:
re.findall('[A-Za-z]', text)

['J',
 'o',
 'h',
 'n',
 'P',
 'R',
 'O',
 'F',
 'J',
 'a',
 'm',
 'e',
 's',
 'S',
 'T',
 'U',
 'D',
 'M',
 'a',
 'c',
 'S',
 'T',
 'U',
 'D']

In [34]:
re.findall('[A-Z]{4}', text)

['PROF', 'STUD', 'STUD']

In [35]:
re.findall('[A-Z][a-z]+', text)

['John', 'James', 'Mac']

#### 정규표현식을 이용한 토큰화

In [37]:
from nltk.tokenize import RegexpTokenizer

In [38]:
text = "Don't be fooled by the dark sounding name, Mr.Jone's Orphanage is as cheery as cheery goes for a pastry shop"

In [40]:
# [\w]+ 는 문자 또는 숫자 한개 이상
tokenizer1 = RegexpTokenizer("[\w]+")

# \s+는 공백기준 토큰화, gaps=true는 정규표현식을 사용해 토큰화한다는 의미
tokenizer2 = RegexpTokenizer('\s+', gaps=True)

<>:2: SyntaxWarning: invalid escape sequence '\w'
<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\w'
<>:5: SyntaxWarning: invalid escape sequence '\s'
C:\Users\dltkd\AppData\Local\Temp\ipykernel_3932\2708969558.py:2: SyntaxWarning: invalid escape sequence '\w'
  tokenizer1 = RegexpTokenizer("[\w]+")
C:\Users\dltkd\AppData\Local\Temp\ipykernel_3932\2708969558.py:5: SyntaxWarning: invalid escape sequence '\s'
  tokenizer2 = RegexpTokenizer('\s+', gaps=True)


In [42]:
print(tokenizer1.tokenize(text))
print(tokenizer2.tokenize(text))

['Don', 't', 'be', 'fooled', 'by', 'the', 'dark', 'sounding', 'name', 'Mr', 'Jone', 's', 'Orphanage', 'is', 'as', 'cheery', 'as', 'cheery', 'goes', 'for', 'a', 'pastry', 'shop']
["Don't", 'be', 'fooled', 'by', 'the', 'dark', 'sounding', 'name,', "Mr.Jone's", 'Orphanage', 'is', 'as', 'cheery', 'as', 'cheery', 'goes', 'for', 'a', 'pastry', 'shop']
